In [21]:
from TwitterAPI import TwitterAPI
import numpy as np
import pandas as pd
from datetime import datetime,timedelta
from pytz import timezone
import json
import matplotlib.pyplot as plt

# Functions

In [15]:
def get_twitter():
    """ 
    Generates twitter connection
    """
    
    #Tokens
    consumer_key='nDx2bQS2DVdXEZd0b8MwaEG2L'
    consumer_secret='BeA1VuBq7hyi01fo4LFD13YAcrDgeYxIJTQgzx4gg8H5fSc3FR'
    access_token='900568210424164352-pKLEmPfNcqyGi3q67pyK3bJYtc5G01S'
    access_token_secret='uGbegX9whUY0FjZS6tgIrDITVOM8Q6tKMODNVPG2sT9Aq'
    
    
    twitter = TwitterAPI(
                   consumer_key,
                   consumer_secret,
                   access_token,
                   access_token_secret)
    return twitter

def get_NumberofFollowers(screen_name):       #does not crash when Exception, but does not correct exceptions either.
    '''
    returns the number of friends the user has
    '''
    try:
        request=twitter.request('users/lookup',{'screen_name':screen_name})
        return request.json()[0]['followers_count']
    except Exception as e:
        print (screen_name +str(request.json()['errors']))
        return np.NaN    

def get_timeline(screen_name,count=200): #should modify this to be more robust
    '''
    retuns a list with the last (count-deleted) tweets from the user
    '''
    try:
        request = twitter.request('statuses/user_timeline',{'screen_name': screen_name,'count': count})
        return request.json()
    except Exception as e:
        print (screen_name +str(request.json()['errors']))
        return np.NaN
def get_TodaysTweets(screen_name,count=200,exclude_replies = True, max_id = None): #should modify this to be more robust
    '''
    retuns a list with the last (count-deleted) tweets from the user
    '''
    try:
        request = twitter.request('statuses/user_timeline',{'screen_name': screen_name,'count': count,
                                                            'exclude_replies':exclude_replies,'max_id':max_id})
        tweet_list = request.json()
        yesterday_DT = datetime.now(timezone('US/Central'))-timedelta(days=1)
        todaysTweets = []
        if tweet_list == []:
            return todaysTweets
        elif not (strToChicago_DT(tweet_list[-1]['created_at']) > yesterday_DT): #last of the 200 was in the last 24h
            todaysTweets =  [t for t in tweet_list if strToChicago_DT(t['created_at']) > yesterday_DT]
            return todaysTweets
        else:
            todaysTweets = tweet_list.copy()
            max_id = todaysTweets[-1]['id']
            todaysTweets += get_TodaysTweets(screen_name,count=200,exclude_replies = True, max_id = max_id)[1:]
            return todaysTweets
    except Exception as e:
        print (screen_name +str(request.json()['errors']))
        return np.NaN

## Helper functions

In [3]:
def strToChicago_DT(string):
    UTC_DT = datetime.strptime(string,'%a %b %d %H:%M:%S +0000 %Y').replace(tzinfo = timezone('UTC'))
    Chicago_DT = UTC_DT.astimezone(timezone('US/Central'))
    return Chicago_DT

# Import Accounts

In [4]:
accounts_df = pd.read_csv('Data/accounts.csv',sep=";")

# Collect Data

In [7]:
twitter = get_twitter()

In [9]:
accounts = accounts_df['Account'] #pd series
todaysTweets = pd.DataFrame(accounts, columns=['Account'])
todaysFollowers = pd.DataFrame(accounts, columns=['Account'])
date=str(datetime.now().date())
todaysTweets[date] = np.NaN
todaysFollowers[date] = np.NaN

In [13]:
todaysTweets[date] = todaysTweets['Account'].apply(get_TodaysTweets)

967235725270843394
967289244753444864
967331041181601793
967237225971728384
967524639869038592
967495477234098176
967460082706345984
967433854922383360
967405615369789440
967374919150452744
967344642457927680
967303589344436224
967255340914847744
967213719942868993
967232658664280066
967411675577176064
967232658664280066
967524639869038592
967495477234098176
967460082706345984
967433854922383360
967405615369789440
967374919150452744
967344642457927680
967303589344436224
967255340914847744
967213719942868993


In [10]:
todaysFollowers[date] = todaysFollowers['Account'].apply(get_NumberofFollowers)

In [11]:
todaysFollowers.head()

,Account,2018-02-24
0,bonappetit,3191238
1,NatGeoFood,35480
2,TwitterFood,838283
3,Fooddotcom,28
4,newfoodeconomy,5903


In [12]:
todaysFollowers.loc[pd.isnull(todaysFollowers[date])] #check nulls

,Account,2018-02-24


# Save Data

In [35]:
todaysFollowers.to_csv('Data/todaysFollowers_'+date+'.csv',sep=';',index=False)

In [43]:
todaysTweets_json = todaysTweets.to_json(orient='records')
with open('Data/todaysTweets_'+date,'w') as fp:
    json.dump(todaysTweets_json,fp)

# Add to Historic Data

In [ ]:
historicFollowers = pd.read_csv('Data/todaysFollowers_all.csv',sep = ';') #load historic Followers

In [ ]:
with open('Data/todaysTweets_all','r') as fp: #load historic tweets
    example = json.load(fp)

In [ ]:
historicFollowers = pd.concat([historicFollowers,todaysFollowers[date]],axis=1) #add todays column

In [ ]:
historicTweets_json = ...
#Todo concat Tweets dfms

In [ ]:
historicFollowers.head()

In [ ]:
historicFollowers.to_csv('Data/todaysFollowers_all.csv',sep=';',index=False) #save

In [ ]:
historicTweets_json = historicTweets.to_json(orient='records')
with open('Data/todaysTweets_all','w') as fp:
    json.dump(historicTweets_json,fp)